# Import thư viện cần thiết

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Import Random Forest Classifier model library
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

Đọc dữ liệu

In [4]:
train = pd.read_csv('../data/train_preprocessed.csv')
train.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,190000.0,0,3,0,1,1,0,0.0,0,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,ID_68de51c94,0.0,0,8,0,1,1,0,0.0,0,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,81,289,16,121,4,1.777778,1.0,121.0,289,4
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,121,1369,16,121,4,1.777778,1.0,121.0,1369,4


In [5]:
test = pd.read_csv('../data/test_preprocessed.csv')
test.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq
0,ID_2f6873615,0.0,0,5,0,1,1,0,0.0,1,...,4,0,16,9,0,1,2.25,0.25,272.25,16
1,ID_1c78846d2,0.0,0,5,0,1,1,0,0.0,1,...,41,256,1681,9,0,1,2.25,0.25,272.25,1681
2,ID_e5442cf6a,0.0,0,5,0,1,1,0,0.0,1,...,41,289,1681,9,0,1,2.25,0.25,272.25,1681
3,ID_a8db26a79,0.0,0,14,0,1,1,1,1.0,0,...,59,256,3481,1,256,0,1.00,0.00,256.00,3481
4,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,0,...,18,121,324,1,0,1,0.25,64.00,121.00,324


In [6]:
ntrain = train.shape[0]
ntest = test.shape[0]

all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.tail()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
33408,ID_a065a7cad,0.0,1,2,1,1,1,0,0.0,0,...,9,100,36,25,4,36.0,0.25,33.0625,100,NaN
33409,ID_1a7c6953b,0.0,0,3,0,1,1,0,0.0,0,...,36,2916,16,36,4,4.0,1.00,36.0000,2916,NaN
33410,ID_07dbb4be2,0.0,0,3,0,1,1,0,0.0,0,...,16,144,16,36,4,4.0,1.00,36.0000,144,NaN
33411,ID_34d2ed046,0.0,0,3,0,1,1,0,0.0,0,...,25,144,16,36,4,4.0,1.00,36.0000,144,NaN
33412,ID_34754556f,0.0,0,3,0,1,1,0,0.0,0,...,36,2601,16,36,4,4.0,1.00,36.0000,2601,NaN


# Quan sát và xử lý dữ liệu đầu vào

## Quan sát dữ liệu

Kích thước dữ liệu.

In [7]:
print('Kích thước tập dữ liệu', all_data.shape, '. Trong đó:\n',
      '- Tập dữ liệu huấn luyện:', train.shape, ';\n',
      '- Tập dữ liệu kiểm thử:', test.shape, '.')

Kích thước tập dữ liệu (33413, 143) . Trong đó:
 - Tập dữ liệu huấn luyện: (9557, 143) ;
 - Tập dữ liệu kiểm thử: (23856, 142) .


Sự chênh lệch dữ liệu là do tập dữ liệu test không chứa biến dự đoán `target`.

Quan sát các biến định tính và định lượng.

In [8]:
numerical_cols = all_data.select_dtypes(include=np.number).columns.tolist()
categorical_cols = list(set(all_data.columns).difference(set(numerical_cols)))

categorical_cols

['idhogar', 'Id']

Quan sát những biến bị thiếu dữ liệu.

In [9]:
# Compute missing values for all columns
all_data.isnull().sum().sort_values(ascending=False)

Target          23856
hogar_mayor         0
parentesco10        0
parentesco11        0
parentesco12        0
                ...  
coopele             0
sanitario1          0
sanitario2          0
sanitario3          0
eviv2               0
Length: 143, dtype: int64

Ngoại trừ biến dự đoán, ta thấy các biến còn lại không có sự xuất hiện dữ liệu bị thiếu.

## Xử lý dữ liệu

### Loại bỏ các thuộc tính không cần thiết

Đầu tiên, cố gắng giảm chiều đối với những biến có thể chuyển từ định tính không thứ tự `one-hot encoding/nominal` sang định tính có thứ tự `ordinal`. Sau khi quan sát ý nghĩa của từng biến, các biến `epared_`, `etecho_` và `eviv_` có cách biểu diễn ý nghĩa mức độ giống nhau, cụ thể đều có 3 giá trị ý nghĩa `bad`, `regular` và `good`. Ta sẽ giảm chiều các biến này và quy ước: `0` - `bad`, `1` - `regular` và `2` - `good`.

In [10]:
def covert_nominal_to_ordinal(df, cols_lst):
    for ele in cols_lst:
        cols_spe = [col for col in df.columns if ele in col]
        spe_df = df[cols_spe].copy()
        spe_df.iloc[:,0] = (spe_df.iloc[:,0] * 0 + spe_df.iloc[:,0] * 1 + spe_df.iloc[:,0] * 2).astype(int)
        df.loc[:,cols_spe[0]] = spe_df[cols_spe[0]]
        df.drop(cols_spe[1:], axis=1, inplace=True)
        df.rename(columns={cols_spe[0]:ele}, inplace=True)
    return df

covert_nominal_to_ordinal(all_data, ['epared', 'etecho', 'eviv'])

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,190000.0,0,3,0,1,1,0,0.0,0,...,100,1849,1,100,0,1.000000,0.00,100.0000,1849,4.0
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,144,4489,1,144,0,1.000000,64.00,144.0000,4489,4.0
2,ID_68de51c94,0.0,0,8,0,1,1,0,0.0,0,...,121,8464,1,0,0,0.250000,64.00,121.0000,8464,4.0
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,81,289,16,121,4,1.777778,1.00,121.0000,289,4.0
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,121,1369,16,121,4,1.777778,1.00,121.0000,1369,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33408,ID_a065a7cad,0.0,1,2,1,1,1,0,0.0,0,...,9,100,36,25,4,36.000000,0.25,33.0625,100,NaN
33409,ID_1a7c6953b,0.0,0,3,0,1,1,0,0.0,0,...,36,2916,16,36,4,4.000000,1.00,36.0000,2916,NaN
33410,ID_07dbb4be2,0.0,0,3,0,1,1,0,0.0,0,...,16,144,16,36,4,4.000000,1.00,36.0000,144,NaN
33411,ID_34d2ed046,0.0,0,3,0,1,1,0,0.0,0,...,25,144,16,36,4,4.000000,1.00,36.0000,144,NaN


Tiếp theo, loại bỏ các thuộc tính có cùng ý nghĩa. Các thuộc tính sau mang cùng ý nghĩa tổng số thành viên trong một hộ gia đình `hogar_total`:
- `tamhog`: size of the household;
- `tamviv`: number of persons living in the household;
- `hhsize`: household size;
- `r4t3`: Total persons in the household.

In [11]:
remove_cols = ['tamhog', 'tamviv', 'hhsize', 'r4t3']

Tiếp theo, loại bỏ các biến suy diễn. Các biến mà thông qua một phép suy diễn trực tiếp từ một biến độc lập khác có thể cung cấp dư thừa thông tin bởi chúng có sẽ có giá trị tương quan rất cao, vì vậy nên xem xét giữ lại một biến trong số chúng:
- `female` có thể suy diễn bằng cách tính $\neg$`male`;
- `area2` có thể suy diễn bằng cách tính $\neg$`area1`;
- `v18q` có thể suy diễn bằng cách ép kiểu `boolean` cho biến `v18q1`;
- `mobilephone` có thể suy diễn bằng cách ép kiểu `boolean` cho biến `qmobilephone`;
- `SQBescolari` có thể suy diễn bằng cách bình phương `escolari`;
- `SQBage` và `agesq` có thể suy diễn bằng cách bình phương `age`;
- `SQBhogar_total` có thể suy diễn bằng cách bình phương `hogar_total`;
- `SQBedjefe` có thể suy diễn bằng cách bình phương `edjefe`;
- `SQBhogar_nin` có thể suy diễn bằng cách bình phương `hogar_nin`;
- `SQBovercrowding` có thể suy diễn bằng cách bình phương `overcrowding`;
- `SQBdependency` có thể suy diễn bằng cách bình phương `dependency`;
- `SQBmeaned` có thể suy diễn bằng cách bình phương `meaneduc`.

In [12]:
remove_cols = np.append(remove_cols, ['female', 'area2', 'v18q', 'mobilephone',
                                      'SQBescolari', 'SQBage', 'agesq', 'SQBhogar_total',
                                      'SQBedjefe', 'SQBhogar_nin', 'SQBovercrowding',
                                      'SQBdependency', 'SQBmeaned'])

In [13]:
all_data.drop(remove_cols, axis=1, inplace=True)

Tiếp theo, quan sát thông thường không thể nhận biết hoàn toàn được những cặp biến có tương quan rất cao, ở đây chúng ta lấy ngưỡng $|$`threshold`$|\geq 0.95$. Cần xác định chúng qua ma trận tương quan.

In [14]:
numerical_cols = all_data.select_dtypes(include=np.number).columns.tolist()

corr_mtr = all_data[numerical_cols[:-1]].corr()
pairs = corr_mtr.where(np.triu(np.ones(corr_mtr.shape), k=1).astype(bool)).stack()

pairs_lst = list(pairs[pairs.abs().gt(0.95)].index)
pairs_lst

[('public', 'coopele')]

- Sau khi xác định được những cặp biến có tương quan rất cao, ta xem xét giữ lại biến đầu tiên xuất hiện trong tập dữ liệu.

In [15]:
for pair in pairs_lst:
    all_data.drop(pair[1], axis=1, inplace=True)

### Thêm các biến suy diễn cung cấp thêm thông tin hữu ích cho bài toán

Cuộc thi hướng đến tiếp cận những hộ gia đình thực sự cần sự hỗ trợ từ chương trình viện trợ xã hội thông qua xác định tiêu chuẩn thu thập của các hộ gia đình. Để mô hình có thêm một số thông tin hữu ích về bài toán, nhóm suy nghĩ nên bổ sung một số thông tin sau:

- Nguồn thu nhập chính chủ động của các hộ gia đình nằm trong độ tuổi từ 19 đến 64 tuổi (cụ thể quan sát từ dữ liệu: từ 20 đến 64 tuổi). Vì vậy nhóm mong muốn cung cấp thêm thông tin về đối tượng là nguồn thu nhập chính chủ động trong hộ gia đình.

In [16]:
# Tính số lượng người có thu nhập nhập chính chủ động trong gia đình
number_adult = all_data['hogar_adul'] - all_data['hogar_mayor']

# Tính tỷ lệ số người có thu nhập chính chủ động trong gia đình
all_data['audult_rate'] = number_adult / all_data['hogar_total']

# Tính tiền thuê nhà dựa trên bình quân thành viên trong gia đình và thành viên mà là người thu nhập chính chủ động
number_adult[number_adult == 0] = 0.5
all_data['rent_per_adult'] = all_data['v2a1']/number_adult
all_data['rent_per_person'] = all_data['v2a1']/all_data['hogar_total']

# Tính tỷ lệ phụ thuộc của mỗi người thu nhập chính chủ động trong mỗi hội gia đình
all_data['dependency_count'] = all_data['hogar_nin'] + all_data['hogar_mayor']
all_data['dependency_per_adult'] = all_data['dependency_count'] / number_adult

number_adult[number_adult == 0.5] = 0
all_data['audult'] = number_adult.astype(int)

- Thông tin về mật độ phòng ốc cũng là thông tin hữu ích khi xem xét vấn đề của bài toán.

In [17]:
# Tính tiền thuê trên mỗi phòng
all_data['rent_per_room'] = all_data['v2a1'] / all_data['rooms']

# Số lượng người trong mỗi phòng
all_data['person_per_bedroom'] = all_data['hogar_total'] / all_data['bedrooms']
all_data['person_per_room'] = all_data['hogar_total'] / all_data['rooms'] # Bởi vì không có diện tích nhà ở, giá trị này có ý nghĩa tương tự số người / 1m^2

- Với những hộ nghèo, việc có các thiết bị điện tử để phục vụ giải trí, học tập và làm việc cũng khó khăn. Các đối tượng này có thể cần được xem xét.

In [18]:
electronic_devices = all_data['computer'] + all_data['qmobilephone'] + all_data['television'] + all_data['v18q1']
all_data['electronic_device_per_person'] = electronic_devices / all_data['hogar_total']

# Xây dựng mô hình học máy

In [19]:
train_processed = all_data[all_data['Target'].notnull()].copy()
train_processed.drop(categorical_cols, axis=1, inplace=True)

In [20]:
y = train_processed['Target']
X = train_processed.iloc[:,:-1]

## Chuẩn bị và quan sát phân bố của tập dữ liệu

Quan sát sự phân bố của biến dự đoán.

In [21]:
y_weights = class_weight.compute_sample_weight('balanced', y, indices=None)

In [22]:
pd.DataFrame(dict(Target = y, Weight = y_weights)).drop_duplicates().sort_values(by = ["Target"]).reset_index(drop = True)

,Target,Weight
0,1.0,3.164570
1,2.0,1.496086
2,3.0,1.976220
3,4.0,0.398474


Chia dữ liệu huấn luyện thành `train` và `test`.

In [24]:
X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(X, y, y_weights, test_size=0.2, random_state=42)

## Xây dựng mô hình

Mô hình sẽ sử dụng sắp tới là Random Forest cho bài toán phân lớp, sử dụng bởi sự hỗ trợ của thư viện `sklearn.ensemble` để khai báo báo mô hình `RandomForestClassifier()`.

`RandomForestClassifier()` là một mô hình ước tính tổng hợp mà chúng sẽ tinh chỉnh một số cây quyết định dựa trên các tập mẫu dữ liệu con khác nhau được sinh từ tập dữ liệu gốc và sử dụng tính trung bình để cải thiện độ chính xác dự đoán và kiểm soát mức độ khớp quá mức.

`RandomForestClassifier()` có một số tham số cần lưu ý thiết lập:
- `n_estimators`: Số lượng cây mà mô hình sẽ tạo ra để học từ dữ liệu huấn luyện. Số lượng cây quá lớn khiến việc học của mô hình trở nên tốn thời gian và có thể bị tình trạng overfitting, ngược lại mô hình có thể học không chính xác dẫn đến underfitting. Giá trị mặc định là `100`.
- `criterion`: Phương thức đo lường chất lượng phân lớp của cây. Có 3 phương tính đó là: `gini`, `entropy` và `log_loss`. Giá trị mặc định là `gini`.
- `max_depth`: Giới hạn độ sâu của cây, thường dùng với tập dữ liệu nhỏ để giảm chi phí tính toán. Giá trị mặc định là `None`, tức các cây trong mô hình sẽ không bị giới hạn.
- `bootstrap`: Bốc mẫu ngẫu nhiên có hoàn lại từ tập dữ liệu nếu mang giá trị `True`, ngược lại, các cây trong mô hình đều có tập mẫu dữ liệu con giống như tập dữ liệu gốc. Giá trị mặc định là `True`.
- `max_samples`: Tham số này có ý nghĩa khi `bootstrap` là `True`, để điều chỉnh số lượng mẫu tối đa của mỗi cây, có thể bằng số nguyên trong đoạn [1, số lượng mẫu] hoặc số thực chỉ tỷ lệ lượng mẫu được lấy trong khoảng (0,1]. Giá trị mặc định là `None`, tức số lượng mẫu tối đa của mỗi cây bằng số lượng mẫu của tập dữ liệu.

In [27]:
# Build structure for Random Forest model using Bagging and MinMaxScaler in a pipeline
rf_pipe = Pipeline([
    ('model', RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42, class_weight='balanced', oob_score=True)),
    ('scaler', MinMaxScaler()),
])

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Create GridSearchCV object for Random Forest model adjusting the number of estimators and criterion{'gini', 'entropy', 'log_loss'}
adjust_para = GridSearchCV(
    
)